<a href="https://colab.research.google.com/github/aldojda/Processing-DataClassification/blob/main/Tarea_3_Multiclasificador_Vectorizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode

In [ ]:
mapeo_correccion = {
    "Ã¡": "á",
    "Ã©": "é",
    "Ã­": "í",
    "Ã³": "ó",
    "Ãº": "ú",
    "Ã¼": "ü",
    "Ã±": "ñ",
    "Ã€": "À",
    "Ãˆ": "È",
    "ÃŒ": "Ì",
    "Ã’": "Ò",
    "Ãš": "Ú",
    "Ãœ": "Ü",
    "Ã‘": "Ñ",
    "Ã‚": "Â",
    "ÃŠ": "Ê",
    "ÃŽ": "Î",
    "Ã”": "Ô",
    "Ã›": "Û",
    "Ã€": "À",
    "Ãˆ": "È",
    "ÃŒ": "Ì",
    "Ã’": "Ò",
    "Ã“": "Ó",
    "Ã”": "Ô",
    "Ã•": "Õ",
    "Ã–": "Ö",
    "Ã‡": "Ç",
    "Ã¡": "á",
    "Ã¢": "â",
    "Ã£": "ã",
    "Ã¤": "ä",
    "Ã¥": "å",
    "Ã¦": "æ",
    "Ã§": "ç",
    "Ã¨": "è",
    "Ã©": "é",
    "Ãª": "ê",
    "Ã«": "ë",
    "Ã¬": "ì",
    "Ã­": "í",
    "Ã®": "î",
    "Ã¯": "ï",
    "Ã°": "ð",
    "Ã±": "ñ",
    "Ã²": "ò",
    "Ã³": "ó",
    "Ã´": "ô",
    "Ãµ": "õ",
    "Ã¶": "ö",
    "Ã·": "÷",
    "Ã¸": "ø",
    "Ã¹": "ù",
    "Ãº": "ú",
    "Ã»": "û",
    "Ã¼": "ü",
    "Ã½": "ý",
    "Ã¾": "þ",
    "Ã¿": "ÿ"
}

In [ ]:
import pandas as pd
import numpy
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('spanish'))
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')
(spanish_stemmer.stem("texto_en_str"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'texto_en_str'

In [ ]:
from unidecode import unidecode
def clean_text(text):
    cleaned_text = unidecode(text)
    # Remove special characters and symbols
    cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', cleaned_text)
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    # Remove leading and trailing spaces
    cleaned_text = cleaned_text.strip()

    # Convert to lowercase
    cleaned_text = cleaned_text.lower()


    # remove stopwords & get tokens

    tokens = word_tokenize(cleaned_text)
    texto_filtrado = [word for word in tokens if word not in stop_words]
    cleaned_text = [spanish_stemmer.stem(word) for word in texto_filtrado ]

    r = ' '.join(texto_filtrado )
    return r

In [ ]:
# prompt: import my files from drive  this path: https://drive.google.com/drive/folders/1WE1l_fW_OQ8Zu0lv5SLve25dngrUZ3gz
from google.colab import drive
drive.mount('/content/drive')

# Change the path to your folder
folder_path = r'drive/MyDrive/HugginFace Multiclass/data/DB_multiclass.csv'
df = pd.read_csv(folder_path,encoding  = "utf-8")
df = df[df["Clasificación Corregida"].isin(['Availability.Stock', 'Deals', 'Delivery.Schedule', 'CX.Staff'])]
df["clean_Comment"] = df["Comment"].replace(mapeo_correccion, regex = True)\
                                        .astype(str).apply(lambda x: clean_text(x))
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Comment,Score,Sentiment,Clasificación Corregida,clean_Comment
7,Xx ambar ya tienen mas de dos meses sin existe...,4.0,Negative,Availability.Stock,xx ambar mas dos meses existencias
8,Visitas del preventa viene cada que lo reporto...,5.0,Negative,CX.Staff,visitas preventa viene cada reporto mas
9,Visitas del personal de ventas,5.0,Negative,CX.Staff,visitas personal ventas
38,Todo bien lo unico que yo solicito es un refri...,5.0,Negative,Availability.Stock,bien unico solicito refrigerador mas grande mu...
70,"Tienen meses con desbasto, ejemplo strongbow r...",3.0,Negative,Availability.Stock,meses desbasto ejemplo strongbow rojo quiten c...


In [ ]:
# dividimos los datos en X, y
X = df['clean_Comment']
y = df['Clasificación Corregida']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('Training Data :', X_train.shape)
print('Testing Data : ', X_test.shape)

Training Data : (13601,)
Testing Data :  (3401,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# conteo de palabras
count_vect = CountVectorizer(analyzer='word')
count_vect.fit(X_train)
xtrain_count =  count_vect.transform(X_train)
xtest_count =  count_vect.transform(X_test)

# palabras
tfidf_vect = TfidfVectorizer(analyzer='word')
xtrain_tfidf =  tfidf_vect.fit_transform(X_train)
xtest_tfidf =  tfidf_vect.transform(X_test)

# ngramas
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', ngram_range=(2,3))
xtrain_tfidf_ngram =  tfidf_vect_ngram.fit_transform(X_train)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

# caracteres
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char')
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.fit_transform(X_train)
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test)

Se realiza el train / test entre distintos modelos con distintos hiper parametros y distintas tecnicas de vectorización

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.datasets import load_iris

models = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('svc', SVC(kernel='linear'))
]

pares = [
    ('conteo', xtrain_count, xtest_count),
    ('palabras', xtrain_tfidf, xtest_tfidf),
    ('ngramas', xtrain_tfidf_ngram, xtest_tfidf_ngram),
    ('caracteres', xtrain_tfidf_ngram_chars, xtest_tfidf_ngram_chars)
]

# Iterate over transformations and models

for datos, X_train, X_test in pares:
    results = []
    print(datos)
    print('-' * 60)
    for model_name, model in models:
        pipeline = Pipeline([
            (model_name, model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        results.append({
            'model': model_name,
            'accuracy': accuracy,
            'report': report
        })

    # Display results
    for result in results:
        print( f"Model: {result['model']}")
        print(f"Accuracy: {result['accuracy']}")
        print(result['report'])
        print('-' * 60)

conteo
------------------------------------------------------------
Model: lr
Accuracy: 0.9261981770067628
                    precision    recall  f1-score   support

Availability.Stock       0.94      0.97      0.95      2067
          CX.Staff       0.89      0.86      0.87       415
             Deals       0.96      0.87      0.91       497
 Delivery.Schedule       0.85      0.86      0.86       422

          accuracy                           0.93      3401
         macro avg       0.91      0.89      0.90      3401
      weighted avg       0.93      0.93      0.93      3401

------------------------------------------------------------
Model: rf
Accuracy: 0.9270802705086739
                    precision    recall  f1-score   support

Availability.Stock       0.95      0.96      0.95      2067
          CX.Staff       0.91      0.84      0.87       415
             Deals       0.95      0.88      0.92       497
 Delivery.Schedule       0.83      0.90      0.87       422

        

In [ ]:
df["Clasificación Corregida"].value_counts()

,count
Clasificación Corregida,
Availability.Stock,10226
Deals,2468
Delivery.Schedule,2169
CX.Staff,2139
